In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
config_file = '/content/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = '/content/frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model, config_file)
classLabels = []
file_name = '/content/labels.txt'
with open(file_name,'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

img = cv2.imread('/content/boy.jpg')
plt.imshow(img)

ClassIndex, confidence, bbox = model.detect(img, confThreshold = 0.5)


font_scale = 3
font = cv2.FONT_HERSHEY_PLAIN
for ClassInd , conf, boxes in zip ( ClassIndex.flatten(), confidence.flatten(), bbox):
    cv2.rectangle(img,boxes,(255,0,0),2)
    cv2.putText(img, classLabels[ClassInd-1],(boxes[0]+40, boxes[1]+40), font, fontScale = font_scale , color = (0,255,0), thickness = 3)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


#video

cap = cv2.VideoCapture('/content/city.mp4')
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('cannot opne the video')
font_scale = 3
font = cv2.FONT_HERSHEY_PLAIN

while True:
    ret , frame = cap.read()
    ClassIndex , confidence, bbox = model.detect(frame, confThreshold=0.55)
    print(ClassIndex)

    if(len(ClassIndex)!=0):
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(),bbox):
            if(ClassInd<=80):
                    cv2.rectangle(img,boxes,(255,0,0),2)
                    cv2.putText(img, classLabels[ClassInd-1],(boxes[0]+10, boxes[1]+40), font, fontScale = font_scale , color = (0,255,0), thickness = 3)
    plt.imshow(cv2.cvtColor(cap, cv2.COLOR_BGR2RGB))
    cv2_imshow(frame)
    if cv2.waitKey(2) & 0xff == ord('q'):
         break
cap.release()
cv2.destroyaLLWindows()

In [ ]:
import cv2
import numpy as np

thres = 0.45  # Threshold to detect object
nms_threshold = 0.2
cap = cv2.VideoCapture(0)
# cap.set(3,1280)
# cap.set(4,720)
# cap.set(10,150)

classNames = []
classFile = 'coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('n').split('n')

    # print(classNames)
    configPath = r"ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
    weightsPath = 'frozen_inference_graph.pb'

    net = cv2.dnn_DetectionModel(weightsPath, configPath)
    net.setInputSize(320, 320)
    net.setInputScale(1.0 / 127.5)
    net.setInputMean((127.5, 127.5, 127.5))
    net.setInputSwapRB(True)

    while True:
        success, img = cap.read()
        classIds, confs, bbox = net.detect(img, confThreshold=thres)
        bbox = list(bbox)
        confs = list(np.array(confs).reshape(1, -1)[0])
        confs = list(map(float, confs))
        # print(type(confs[0]))
        # print(confs)

        indices = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_threshold)
        # print(indices)

        for i in indices:
            i = i[0]
            box = bbox[i]
            x, y, w, h = box[0], box[1], box[2], box[3]
            cv2.rectangle(img, (x, y), (x + w, h + y), color=(0, 255, 0), thickness=2)
            cv2.putText(img, classNames[classIds[i][0] - 1].upper(), (box[0] + 10, box[1] + 30),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Output",img)
        cv2.waitKey(1)

https://drive.google.com/drive/folders/1GrFlJNaQ9eAKcFo9MdBa7lgC0xxl-PRw

https://www.computervision.zone/topic/nms-code/

https://github.com/pjreddie/darknet/blame/master/data/coco.names or https://github.com/Ranjan2104/Object-Detection-OpenCV/blob/main/coco.names

https://www.youtube.com/watch?v=HXDD7-EnGBY&t=39s

In [ ]:
import cv2
thres = 0.45 # Threshold to detect object

cap = cv2.VideoCapture(1)
cap.set(3,1280)
cap.set(4,720)
cap.set(10,70)

classNames= []
classFile = ‘coco.names’
with open(classFile,’rt’) as f:
classNames = f.read().rstrip(‘n’).split(‘n’)

configPath = ‘ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt’
weightsPath = ‘frozen_inference_graph.pb’

net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

while True:
    success,img = cap.read()
    classIds, confs, bbox = net.detect(img,confThreshold=thres)
    print(classIds,bbox)

    if len(classIds) != 0:
        for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
            cv2.rectangle(img,box,color=(0,255,0),thickness=2)
            cv2.putText(img,classNames[classId-1].upper(),(box[0]+10,box[1]+30),
            cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.putText(img,str(round(confidence*100,2)),(box[0]+200,box[1]+30),
            cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

    cv2.imshow(“Output”,img)
    cv2.waitKey(1)